In [39]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Function to scrape the natural gas prices table
def scrape_gas_prices(url):
    # Send a request to the webpage
    response = requests.get(url)
    
    if response.status_code != 200:
        print("Failed to retrieve the page.")
        return None
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the correct table (This might vary depending on the page structure)
    # Here we try using Pandas first to extract tables directly from HTML
    try:
        tables = pd.read_html(response.text)
        if len(tables) > 0:
            # Iterate over all tables found on the page
            for idx, table in enumerate(tables):
                print(f"Table {idx}:")
                print(table.head())  # Print the first few rows for inspection
                
                # Check for correct headers in the table (for example "Year" column)
                if "Year" in table.columns:
                    print("Correct table found!")
                    return table  # Return the correct table
    except Exception as e:
        print(f"Error using pandas.read_html: {e}")
    
    # If pandas.read_html fails, fallback to BeautifulSoup
    try:
        # Find the table in BeautifulSoup
        table = soup.find('table')  # You may need to inspect the page and adjust this
        
        # Extract rows from the table
        data = []
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all(['td', 'th'])  # td for data, th for header
            cols = [ele.text.strip() for ele in cols]  # Clean the data
            if len(cols) == 13:  # Ensure the correct number of columns (12 months + 1 year)
                data.append(cols)
        
        # Convert data to a DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])  # First row is header
        return df
    except Exception as e:
        print(f"Error scraping with BeautifulSoup: {e}")
        return None

# URL of the webpage containing the data
url = 'https://www.eia.gov/dnav/ng/hist/rngwhhdm.htm'  # Example URL, replace with actual URL if different

# Fetch the data
df = scrape_gas_prices(url)

# If the dataframe was successfully created, clean and save it
if df is not None:
    # Remove rows with missing or irrelevant data
    #df = df.dropna()  # Remove rows with missing data
    
    # Clean the dataframe if needed, e.g., by removing any extra columns or rows
    # This step depends on how clean the extracted table is
    
    # Save to a CSV file
    df.to_csv('natural_gas_prices.csv', index=False)
    print("Data saved to 'natural_gas_prices.csv'")
else:
    print("No data to save.")


Table 0:
                                                   0
0  View History: Daily Weekly Monthly Annual Down...
1  Henry Hub Natural Gas Spot Price (Dollars per ...
Table 1:
                                           0                         1
0  View History: Daily Weekly Monthly Annual  Download Data (XLS File)
Table 2:
                                           0
0  View History: Daily Weekly Monthly Annual
Table 3:
               0   1      2   3       4   5        6   7       8
0  View History: NaN  Daily NaN  Weekly NaN  Monthly NaN  Annual
Table 4:
     Year   Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct   Nov  \
0  1997.0  3.45  2.15  1.89  2.03  2.25  2.20  2.19  2.49  2.88  3.07  3.01   
1  1998.0  2.09  2.23  2.24  2.43  2.14  2.17  2.17  1.85  2.02  1.91  2.12   
2  1999.0  1.85  1.77  1.79  2.15  2.26  2.30  2.31  2.80  2.55  2.73  2.37   
3     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4  2000.0  2.42  2.66  2.79  3.04  3.59

C:\Users\Prince\AppData\Local\Temp\ipykernel_11676\1912775118.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [40]:
df = pd.read_csv('natural_gas_prices.csv')

In [41]:
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1997.0,3.45,2.15,1.89,2.03,2.25,2.20,2.19,2.49,2.88,3.07,3.01,2.35
1,1998.0,2.09,2.23,2.24,2.43,2.14,2.17,2.17,1.85,2.02,1.91,2.12,1.72
2,1999.0,1.85,1.77,1.79,2.15,2.26,2.30,2.31,2.80,2.55,2.73,2.37,2.36
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000.0,2.42,2.66,2.79,3.04,3.59,4.29,3.99,4.43,5.06,5.02,5.52,8.90
